In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/Test Data.csv
/kaggle/input/dataset/Training Data.csv
/kaggle/input/dataset/Sample Prediction Dataset.csv


In [2]:
train_data=pd.read_csv("../input/dataset/Training Data.csv")
test_data=pd.read_csv("../input/dataset/Test Data.csv")

In [3]:
train_data.shape

(252000, 13)

In [4]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [5]:
train_data["profession"]=pd.factorize(train_data.profession)[0]
train_data["city"]=pd.factorize(train_data.city)[0]
train_data["state"]=pd.factorize(train_data.state)[0]
train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]

In [6]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,0,0,0,0,0,0,3,13,0
1,2,7574516,40,10,0,0,0,1,1,1,9,13,0
2,3,3991815,66,4,1,0,0,2,2,2,4,10,0
3,4,6256451,41,2,0,0,1,1,3,3,2,12,1
4,5,5768871,47,11,0,0,0,3,4,4,3,14,1


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [8]:
y_labels=train_data["risk_flag"]
x_features=train_data.drop(["risk_flag"],axis=1)

In [9]:
def model_fit(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = LogisticRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_pred))
    
    

In [10]:
model_fit(x_features,y_labels)

0.8759325396825397


In [11]:
#random sampling
data1=train_data[train_data.risk_flag==0]
data2=train_data[train_data.risk_flag==1]
data3=data1.sample(n=40000)
random_train_data=data3.append(data2)
random_train_data.shape

(70996, 13)

In [12]:
random_y_labels=random_train_data["risk_flag"]
random_x_features=random_train_data.drop(["risk_flag"],axis=1)

In [13]:
model_fit(random_x_features,random_y_labels)

0.5654225352112676


In [14]:
#stratify sampling
def stratify_model_fit(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.2, random_state=42)
    model = LogisticRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(accuracy_score(y_test, y_pred))

In [15]:
stratify_model_fit(x_features,y_labels)

0.8770039682539682


In [16]:
stratify_model_fit(random_x_features,random_y_labels)

0.5633802816901409


In [17]:
#reserviour sampling 
import random
def generator(max):
    number = 1
    while number < max:
        number += 1
        yield number
# Create as stream generator
stream = generator(10000)
# Doing Reservoir Sampling from the stream
k=5
reservoir = []
for i, element in enumerate(stream):
    if i+1<= k:
        reservoir.append(element)
    else:
        probability = k/(i+1)
        if random.random() < probability:
            # Select item in stream and remove one of the k items already selected
             reservoir[random.choice(range(0,k))] = element
print(reservoir)

[7325, 5195, 3822, 8966, 9802]


In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
from imblearn.over_sampling import *

In [19]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks(return_indices=True, ratio='majority')
X_tl, y_tl, id_tl = tl.fit_sample(x_features,y_labels)

TypeError: __init__() got an unexpected keyword argument 'return_indices'

In [20]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(x_features,y_labels)

TypeError: __init__() got an unexpected keyword argument 'ratio'

Variations of Oversampling techniques:
RandomOverSampler,
SMOTE,
BorderLineSMOTE,
KMeansSMOTE,
SVMSMOTE,
ADASYN,
SMOTENC,


In [24]:
sm = RandomOverSampler(random_state=42)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [25]:
model_fit(X_res, y_res)

0.5096491029614715


In [26]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [27]:
model_fit(X_res, y_res)

0.5114703287255944


In [28]:
sm = BorderlineSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [29]:
model_fit(X_res, y_res)

0.512522341123504


In [30]:
sm = KMeansSMOTE(random_state=42, cluster_balance_threshold=0.1)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [31]:
model_fit(X_res, y_res)

0.6524439215863715


In [ ]:
sm = SVMSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [ ]:
model_fit(X_res, y_res)

In [ ]:
sm = ADASYN(random_state=42)
X_res, y_res = sm.fit_resample(x_features,y_labels)

In [ ]:
model_fit(X_res, y_res)